# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [17]:
#importando bibliotecas
%matplotlib inline
import pandas as pd
import numpy as np
from emoji import UNICODE_EMOJI
import math as math
from scipy import stats
import matplotlib.pyplot as plt
import math

In [18]:
#Leitura do arquivo PEPSI - excel 
pepsi = pd.read_excel('tweets_pepsi_201809092234.xlsx')
pepsi.head(8)

,Treinamento,Critério
0,como fazer um quadcopter caseiro elétrico (dro...,Irrelevante
1,rt @paradise_wang: 2015👶\n2018🤵\n\no dream nig...,Relevante
2,@vaijuanc a britney é parceira da pepsi a déca...,Irrelevante
3,mais uma madrugada no chao do banheiro chorand...,Relevante
4,fetos abortados na pepsi é saudavel e recomendo,Irrelevante
5,rt @paradise_wang: 2015👶\n2018🤵\n\no dream nig...,Relevante
6,"rt @aaybrunomars: 7.09/8.09 | denver, colorado...",Relevante
7,@r_gonza @pepsibr prefiro pepsi do que coca-co...,Relevante


In [51]:
#criando função que limpa os tweets
def limpa_tweet(coluna):
    coluna.str.replace(".","")
    coluna.str.replace(",","")
    coluna.str.replace(":","")
    coluna.str.replace(";","")
    coluna.str.replace("@","")
    coluna.str.replace("rt","")
    coluna.str.replace("_","")
    coluna.str.replace("?","")
    coluna.str.replace("!","")
    coluna.str.replace("(","")
    coluna.str.replace(")","")
    coluna.str.replace("/","")
    coluna.str.replace("+","")
    coluna.str.replace("-"," ")
    coluna.str.replace("=","")
    coluna.str.replace("|","")
    coluna.str.replace("#","")
    coluna.str.replace("*","")
    coluna.str.replace("$","")
    coluna.str.replace("{","")
    coluna.str.replace("}","")
    coluna.str.replace("[","")
    coluna.str.replace("]","")
    coluna.str.replace('"',"")
    coluna.str.replace('\n',"")
    coluna.str.replace("_","")
    coluna.str.replace(" por ", " ")
    coluna.str.replace(" do ", " ")
    coluna.str.replace(" dos ", " ")
    coluna.str.replace(" da ", " ")
    coluna.str.replace(" das ", " ")
    coluna.str.replace(" de ", " ")
    coluna.str.replace(" no ", " ")
    coluna.str.replace(" nos ", " ")
    coluna.str.replace(" na ", " ")
    coluna.str.replace(" nas ", " ")
    coluna.str.replace(" é ", " ")
    coluna.str.replace(" e ", " ")
    coluna.str.replace(" a ", " ")
    coluna.str.replace(" à ", " ")
    coluna.str.replace(" á ", " ")
    coluna.str.replace(" ao ", " ")
    coluna.str.replace(" aos ", " ")
    coluna.str.replace(" os ", " ")
    coluna.str.replace(" as ", " ")
    coluna.str.replace(" o ", " ")
    coluna.str.replace(" ò ", " ")
    coluna.str.lower()
    return coluna

limpa_tweet(pepsi["Treinamento"])
pepsi.head()

,Treinamento,Critério
0,como fazer um quadcopter caseiro elétrico dron...,Irrelevante
1,paradisewang 2015 👶 2018 🤵 o dream night foi ...,Relevante
2,vaijuanc britney parceira pepsi décadas kkk qu...,Irrelevante
3,mais uma madrugada chao banheiro chorando com ...,Relevante
4,fetos aboados pepsi saudavel recomendo,Irrelevante


In [57]:
#criando funcao que coloca espaços entre emojis
def espaco_emoji(coluna):
    lista_espacada_emoji = []
    for tweet in coluna:
        nada = ''
        for palavra in tweet:
            if palavra in UNICODE_EMOJI:
                nada = nada + " " + palavra + " "
            else:
                nada += palavra
        lista_espacada_emoji.append(nada)
    return lista_espacada_emoji


pepsi["Treinamento"] = espaco_emoji(pepsi["Treinamento"])
pepsi.head()

,Treinamento,Critério
0,como fazer um quadcopter caseiro elétrico dron...,Irrelevante
1,paradisewang 2015 👶 2018 🤵 o dream night ...,Relevante
2,vaijuanc britney parceira pepsi décadas kkk qu...,Irrelevante
3,mais uma madrugada chao banheiro chorando com ...,Relevante
4,fetos aboados pepsi saudavel recomendo,Irrelevante


In [21]:
#separando tweets relevantes do total
pepsi_relevante = pepsi[(pepsi["Critério"]=="Relevante")]
pepsi_relevante.head()

,Treinamento,Critério
1,paradisewang 2015 👶 2018 🤵 o dream night foi ...,Relevante
3,mais uma madrugada chao banheiro chorando com ...,Relevante
5,paradisewang 2015 👶 2018 🤵 o dream night foi ...,Relevante
6,aaybrunomars 709809 denver colorado 🇺🇸 24kma...,Relevante
7,rgonza pepsibr prefiro pepsi que coca cola exe...,Relevante


In [22]:
#separando tweets irrelevantes do total
pepsi_irrelevante = pepsi[(pepsi["Critério"]=="Irrelevante")]
pepsi_irrelevante.head()

,Treinamento,Critério
0,como fazer um quadcopter caseiro elétrico dron...,Irrelevante
2,vaijuanc britney parceira pepsi décadas kkk qu...,Irrelevante
4,fetos aboados pepsi saudavel recomendo,Irrelevante
9,bspearsbr 📢 notícia 📢 randee divulga em se...,Irrelevante
11,eu tô muito podre só bebi pepsi ontem,Irrelevante


In [23]:
#listando palavras que compõem os tweets relevantes
lista_palavras_relevantes = []
for frase_relevante in pepsi_relevante["Treinamento"]:
    for palavra_relevante in frase_relevante.split():
        lista_palavras_relevantes.append(palavra_relevante)
print(lista_palavras_relevantes)

['paradisewang', '2015', '👶', '2018', '🤵', 'o', 'dream', 'night', 'foi', 'um', 'web', 'drama', 'pra', 'fazer', 'gente', 'rir', 'produção', 'mas', 'essa', 'propaganda', 'pepsi', 'jackson', 'ma…', 'mais', 'uma', 'madrugada', 'chao', 'banheiro', 'chorando', 'com', 'minha', 'fiel', 'companheira', 'garrafa', 'pepsi', 'paradisewang', '2015', '👶', '2018', '🤵', 'o', 'dream', 'night', 'foi', 'um', 'web', 'drama', 'pra', 'fazer', 'gente', 'rir', 'produção', 'mas', 'essa', 'propaganda', 'pepsi', 'jackson', 'ma…', 'aaybrunomars', '709809', 'denver', 'colorado', '🇺🇸', '24kmagicworldtour', '📍', 'pepsi', 'center', '📍', 'capacidade', '20000', '📍', 'horário', 'previsto', '2115', 'hora…', 'rgonza', 'pepsibr', 'prefiro', 'pepsi', 'que', 'coca', 'cola', 'exemplo', 'mas', 'um', 'saco', 'tem', 'que', 'forcejar', 'pra', 'abrir', 'uma', 'garrafa', 'que', 'chega', 'estufar', 'paradisewang', '2015', '👶', '2018', '🤵', 'o', 'dream', 'night', 'foi', 'um', 'web', 'drama', 'pra', 'fazer', 'gente', 'rir', 'produção',

In [24]:
#listando palavras que compõem os tweets irrelevantes
lista_palavras_irrelevantes = []
for frase_irrelevante in pepsi_irrelevante["Treinamento"]:
    for palavra_irrelevante in frase_irrelevante.split():
        lista_palavras_irrelevantes.append(palavra_irrelevante)
print(lista_palavras_irrelevantes)

['como', 'fazer', 'um', 'quadcopter', 'caseiro', 'elétrico', 'drone', 'com', 'latinha', 'refrigerantepepsi', '080918', 'httpstcomv5e2vlqmc', 'httpstconnvnrwwfhw', 'vaijuanc', 'britney', 'parceira', 'pepsi', 'décadas', 'kkk', 'qual', 'novidade', 'fetos', 'aboados', 'pepsi', 'saudavel', 'recomendo', 'bspearsbr', '📢', 'notícia', '📢', 'randee', 'divulga', 'em', 'seu', 'instagram', 'foto', 'original', 'pepsi', '2018', 'httpstcod2wjmkdndv', 'eu', 'tô', 'muito', 'podre', 'só', 'bebi', 'pepsi', 'ontem', 'gostei', 'um', 'vídeo', 'youtube', 'httpstcor7pvsiwimk', 'cola', 'vs', 'pepsi', 'pae', '2', 'httpstcopjyproe4wd', 'sambote', 'velho', 'barreiro', 'com', 'pepsi', 'banho', 'né', 'julyabergamasco', 'simmmm', 'vou', 'ter', 'que', 'ensinar', 'pepsi', 'tirar', 'fotos', 'pra', 'mim', 'kkkk', 'carlosbolsonaro', 'não', 'foi', 'lei', 'proibição', 'arma', 'fogo', 'que', 'salvou', 'bolsonaro', 'foi', 'deus', 'estado', 'onde', 'tentaram', 'matar', 'meu', 'pai', 'também…', 'anterogreco', 'admiro', 'sua', '

In [25]:
#listando palavras que compõem todos os tweets
lista_palavras = []
for frase in pepsi["Treinamento"]:
    for palavra in frase.split():
        lista_palavras.append(palavra)
print(lista_palavras)

['como', 'fazer', 'um', 'quadcopter', 'caseiro', 'elétrico', 'drone', 'com', 'latinha', 'refrigerantepepsi', '080918', 'httpstcomv5e2vlqmc', 'httpstconnvnrwwfhw', 'paradisewang', '2015', '👶', '2018', '🤵', 'o', 'dream', 'night', 'foi', 'um', 'web', 'drama', 'pra', 'fazer', 'gente', 'rir', 'produção', 'mas', 'essa', 'propaganda', 'pepsi', 'jackson', 'ma…', 'vaijuanc', 'britney', 'parceira', 'pepsi', 'décadas', 'kkk', 'qual', 'novidade', 'mais', 'uma', 'madrugada', 'chao', 'banheiro', 'chorando', 'com', 'minha', 'fiel', 'companheira', 'garrafa', 'pepsi', 'fetos', 'aboados', 'pepsi', 'saudavel', 'recomendo', 'paradisewang', '2015', '👶', '2018', '🤵', 'o', 'dream', 'night', 'foi', 'um', 'web', 'drama', 'pra', 'fazer', 'gente', 'rir', 'produção', 'mas', 'essa', 'propaganda', 'pepsi', 'jackson', 'ma…', 'aaybrunomars', '709809', 'denver', 'colorado', '🇺🇸', '24kmagicworldtour', '📍', 'pepsi', 'center', '📍', 'capacidade', '20000', '📍', 'horário', 'previsto', '2115', 'hora…', 'rgonza', 'pepsibr', '

In [26]:
#frequencia das palavras que compõem os tweets relevantes
pd.Series(lista_palavras_relevantes).value_counts()

pepsi                 191
um                     60
2018                   59
pra                    55
mas                    54
gente                  50
essa                   48
foi                    48
propaganda             47
fazer                  46
web                    45
ma…                    45
jackson                45
dream                  45
drama                  45
2015                   45
o                      45
🤵                      45
rir                    45
paradisewang           45
produção               45
night                  45
👶                      45
uma                    42
que                    39
😂                      35
em                     34
foto                   30
📢                      28
com                    27
                     ... 
httpstco0996lyuicz      1
abrir                   1
resto                   1
~                       1
👼                       1
rihanna                 1
atestado                1
bcaalian    

In [27]:
#frequencia das palavras que compõem os tweets irrelevantes
pd.Series(lista_palavras_irrelevantes).value_counts()

pepsi            71
que              41
não              24
eu               24
um               19
com              16
cola             15
uma              15
coca             13
minha            12
😯                12
em                9
se                8
nao               8
ela               8
ser               8
pra               7
só                7
mas               7
tem               7
foi               6
esse              6
mais              6
sua               6
q                 6
ai                6
para              6
como              6
me                5
to                5
                 ..
black             1
restaurante       1
agua              1
600               1
kkkkkkkkkkkkk     1
parece            1
velho             1
também…           1
peito…            1
milhares          1
randee            1
vem               1
fiquei            1
makrosa           1
homens            1
acho              1
youtube           1
mensagens         1
tentaram          1


In [28]:
#frequencia das palavras totais
pd.Series(lista_palavras).value_counts()

pepsi                  262
que                     80
um                      79
pra                     62
2018                    61
mas                     61
uma                     57
foi                     54
gente                   53
essa                    53
fazer                   50
propaganda              47
o                       46
night                   45
2015                    45
eu                      45
👶                       45
dream                   45
🤵                       45
rir                     45
web                     45
produção                45
ma…                     45
paradisewang            45
jackson                 45
drama                   45
em                      43
com                     43
😂                       35
não                     33
                      ... 
mccasquinha              1
tome                     1
pq                       1
causa                    1
muita                    1
decider                  1
~

In [29]:
#testando o length obtido acima
lista_palavras_sem_rep = []
for palavra in lista_palavras:
    if palavra not in lista_palavras_sem_rep:
        lista_palavras_sem_rep.append(palavra)

print(len(lista_palavras_sem_rep))


1334


In [30]:
#criando funcao classificador de palavras
len_rel = len(lista_palavras_relevantes) #3739
len_irrel = len(lista_palavras_irrelevantes) #1511
total_sem_rep = len(lista_palavras_sem_rep) #length do value counts do total de palavras, mas também obtido pelo codigo da celula acima

relevantes = pd.Series(lista_palavras_relevantes).value_counts()
irrelevantes = pd.Series(lista_palavras_irrelevantes).value_counts()

def classificador_palavra(palavra):
    if palavra in relevantes and palavra in irrelevantes:
        P_rel =  (relevantes[palavra] + 1)/(len_rel + total_sem_rep)
        P_irrel = (irrelevantes[palavra] + 1)/(len_irrel + total_sem_rep)
        
    if palavra in relevantes and palavra not in irrelevantes:
        P_rel = (relevantes[palavra] + 1)/(len_rel + total_sem_rep)
        P_irrel = (0 + 1)/(len_irrel + total_sem_rep)
    
    if palavra not in relevantes and palavra in irrelevantes:
        P_rel = (0 + 1)/(len_rel + total_sem_rep)
        P_irrel = (irrelevantes[palavra] + 1)/(len_irrel + total_sem_rep)
    
    if palavra not in relevantes and palavra not in irrelevantes:
        P_rel = (0 + 1)/(len_rel + total_sem_rep)
        P_irrel = (0 + 1)/(len_irrel + total_sem_rep)
    
    return[P_rel, P_irrel]

print(classificador_palavra("pepsi"))
print(classificador_palavra("pepsi")[0])

[0.043146067415730335, 0.027767065175472425]
0.043146067415730335


In [31]:
#calculando probabilidade de ser relevante e irrelevnate
P_relevante = len(pepsi_relevante["Critério"])/len(pepsi["Critério"])
P_irrelevante = len(pepsi_irrelevante["Critério"])/len(pepsi["Critério"])
print(P_relevante)
print(P_irrelevante)

0.6866666666666666
0.31333333333333335


In [32]:
#criando classificador de frases
def classificador_tweet(tweet):
    P_rel = 1
    P_irrel = 1
    lista_tweet = tweet.split()
    for palavra in lista_tweet:
        P_rel *= classificador_palavra(palavra)[0]
        P_irrel *= classificador_palavra(palavra)[1]
    P_rel = math.log(P_rel * P_relevante)
    P_irrel = math.log(P_irrel * P_irrelevante)
    if P_rel > P_irrel:
        return "Relevante"
        #return "O tweet tem grandes chances de ser relevante."
    elif P_rel < P_irrel:
        return "Irrelevante"
        #return "O tweet tem grandes chances de ser irrelevante."
    else:
        return "O tweet ta bugado."

#testando para um tweet aleatório
print(classificador_tweet("eu amo pepsi"))

Relevante


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [33]:
#lendo o arquivo Teste do excel da pepsi
pepsi_teste = pd.read_excel('tweets_pepsi_201809092234.xlsx', sheet_name=1)
pepsi_teste.head()

,Teste,Critério
0,@olavinha88 @pombococo @craudin_ @mongolasc @r...,Irrelevante
1,"um camarão com macarrão no alho , uma pepsi dp...",Irrelevante
2,@aguadeunicornio @_mirianpereira você salvou u...,Irrelevante
3,passei o dia todo hj estressada querendo uma c...,Relevante
4,sua pepsi maravilhosa foi o centro da discussã...,Relevante


In [53]:
#limpando os tweets Teste 
limpa_tweet(pepsi_teste["Teste"])
pepsi_teste.head()

,Teste,Critério,Classificação
0,olavinha88 pombococo craudin mongolasc rollerg...,Irrelevante,Irrelevante
1,um camarão com macarrão alho uma pepsi dps un...,Irrelevante,Irrelevante
2,aguadeunicornio mirianpereira você salvou uma ...,Irrelevante,Irrelevante
3,passei dia todo hj estressada querendo uma coc...,Relevante,Relevante
4,sua pepsi maravilhosa foi centro discussão htt...,Relevante,Irrelevante


In [59]:
#colocando espacos entre os emojis dos tweets Teste
pepsi_teste["Teste"] = espaco_emoji(pepsi_teste["Teste"])
pepsi_teste.head()

,Teste,Critério,Classificação
0,olavinha88 pombococo craudin mongolasc rollerg...,Irrelevante,Irrelevante
1,um camarão com macarrão alho uma pepsi dps un...,Irrelevante,Irrelevante
2,aguadeunicornio mirianpereira você salvou uma ...,Irrelevante,Irrelevante
3,passei dia todo hj estressada querendo uma coc...,Relevante,Relevante
4,sua pepsi maravilhosa foi centro discussão htt...,Relevante,Irrelevante


In [36]:
#filtrando os tweets teste de acordo com o critério relevante
pepsi_teste_relevante = pepsi_teste[(pepsi_teste["Critério"]=="Relevante")]
#pepsi_teste_relevante.head()

In [37]:
#filtrando os tweets teste de acordo com o critério irrelevante
pepsi_teste_irrelevante = pepsi_teste[(pepsi_teste["Critério"]=="Irrelevante")]
#pepsi_teste_irrelevante.head()

In [38]:
#classificando os tweets teste usando a funcao criada anteriormente
classificacao_teste = []
for tweet in pepsi_teste["Teste"]:
    classificacao_teste.append(classificador_tweet(tweet))

print(len(classificacao_teste))
print(classificacao_teste)  

200
['Irrelevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Relevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Relevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Relevante', 'Irrelevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Relevante', 'Irrelevante', 'Relevante', 'Irrelevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Relevante', 'Irrelevante', 'Relevante', 'Relevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Irrelevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Irrelevante', 'Relevante', 'Irrelevante', 'Relevante', 'Irrelevante', 'Relevante', 'Irrelevante', 'Relevante', 

In [39]:
#adicionando a classificação em relevante e irrelevante pelo bot ao dataframe inicial do pepsi Teste
pepsi_teste["Classificação"] = pd.Series(classificacao_teste)
pepsi_teste.head(10)

,Teste,Critério,Classificação
0,olavinha88 pombococo craudin mongolasc rollerg...,Irrelevante,Irrelevante
1,um camarão com macarrão alho uma pepsi dps un...,Irrelevante,Irrelevante
2,aguadeunicornio mirianpereira você salvou uma ...,Irrelevante,Irrelevante
3,passei dia todo hj estressada querendo uma coc...,Relevante,Relevante
4,sua pepsi maravilhosa foi centro discussão htt...,Relevante,Irrelevante
5,observaldo eu falei repito q vc so fala merda ...,Irrelevante,Irrelevante
6,anawithann gengibraotwistprovando novo sabor ...,Relevante,Irrelevante
7,btsare7 como esquecer dia que jimin mesmo com...,Irrelevante,Relevante
8,felipevangeli esquerda tudo burra inteligente...,Irrelevante,Relevante
9,aventurasjs denúncia pepsi agora adoçada com ...,Relevante,Relevante


In [40]:
#verificando relevância dos tweets classificados por nós
pepsi_teste["Critério"].value_counts()

Relevante      153
Irrelevante     47
Name: Critério, dtype: int64

In [41]:
#verificando relevância dos tweets classificados pelo bot
pepsi_teste["Classificação"].value_counts()

Irrelevante    111
Relevante       89
Name: Classificação, dtype: int64

In [42]:
#Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
pepsi_positivos_falsos = pepsi_teste[(pepsi_teste["Critério"]=="Irrelevante")&(pepsi_teste["Classificação"]=="Relevante")]
pepsi_positivos_falsos.head()

,Teste,Critério,Classificação
7,btsare7 como esquecer dia que jimin mesmo com...,Irrelevante,Relevante
8,felipevangeli esquerda tudo burra inteligente...,Irrelevante,Relevante
18,paradisewang 2015 👶 2018 🤵 o dream night foi ...,Irrelevante,Relevante
19,tomei pepsi gelada mas cerveja tava quente 💔,Irrelevante,Relevante
21,denúncia pepsi agora adoçada com fetos aboados...,Irrelevante,Relevante


In [43]:
a = len(pepsi_positivos_falsos)/len(pepsi_teste)
print(a)

0.055


In [44]:
#Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
pepsi_positivos_verdadeiros = pepsi_teste[(pepsi_teste["Critério"]=="Relevante")&(pepsi_teste["Classificação"]=="Relevante")]
pepsi_positivos_verdadeiros.head()

,Teste,Critério,Classificação
3,passei dia todo hj estressada querendo uma coc...,Relevante,Relevante
9,aventurasjs denúncia pepsi agora adoçada com ...,Relevante,Relevante
12,paradisewang 2015 👶 2018 🤵 o dream night foi ...,Relevante,Relevante
13,jhoseokfave jimin pergunta qual bebida mais f...,Relevante,Relevante
14,paradisewang 2015 👶 2018 🤵 o dream night foi ...,Relevante,Relevante


In [45]:
b = len(pepsi_positivos_verdadeiros)/len(pepsi_teste)
print(b)

0.39


In [46]:
#Porcentagem de negativos falsos (marcado como não relevante e são relevantes)
pepsi_negativos_falsos = pepsi_teste[(pepsi_teste["Critério"]=="Relevante")&(pepsi_teste["Classificação"]=="Irrelevante")]
pepsi_negativos_falsos.head()

,Teste,Critério,Classificação
4,sua pepsi maravilhosa foi centro discussão htt...,Relevante,Irrelevante
6,anawithann gengibraotwistprovando novo sabor ...,Relevante,Irrelevante
10,coloursuga eu achava pepsi melhor mas como eu ...,Relevante,Irrelevante
11,essa pepsi gengibre maravilhosa pena que ediçã...,Relevante,Irrelevante
22,meu deus céu tomar uma latinha pepsi depois fi...,Relevante,Irrelevante


In [47]:
c = len(pepsi_negativos_falsos)/len(pepsi_teste)
print(c)

0.375


In [48]:
#Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
pepsi_negativos_verdadeiros = pepsi_teste[(pepsi_teste["Critério"]=="Irrelevante")&(pepsi_teste["Classificação"]=="Irrelevante")]
pepsi_negativos_verdadeiros.head()

,Teste,Critério,Classificação
0,olavinha88 pombococo craudin mongolasc rollerg...,Irrelevante,Irrelevante
1,um camarão com macarrão alho uma pepsi dps un...,Irrelevante,Irrelevante
2,aguadeunicornio mirianpereira você salvou uma ...,Irrelevante,Irrelevante
5,observaldo eu falei repito q vc so fala merda ...,Irrelevante,Irrelevante
15,fui shop pra comer so essencial eu comprei um ...,Irrelevante,Irrelevante


In [49]:
d = len(pepsi_negativos_verdadeiros)/len(pepsi_teste)
print(d)

0.18


In [50]:
semelhanca = b + d
print(semelhanca)

0.5700000000000001


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


   A partir da proposta de criar uma inteligência que conseguisse extrair emoções de tweets e classificá-las, o grupo se engajou no desafio de a partir de postagens sobre produtos, identificar emoções de usuários reais. A amostra obtida foi de 500 tweets, sendo 300 para o treinamento do programa e 200 para realizar o teste de performance.

   Com isso os processos realizados foram: separação manual de tweets relevantes e irrelevantes, “limpeza” dos tweets, retirando informações sem valor de análise, contagem e classificação de palavras como relevantes ou irrelevantes. Juntamente com suas respectivas probabilidades em cada caso, e por fim um teste com os 200 tweets destinados a essa finalidade.

   Em termos qualitativos de análise dos resultados o programa foi capaz de processar informações sobre os tweets recebidos e a partir delas diagnosticar se ele seria relevante ou irrelevante  com um índice de precisão maior que a média. Em casos de sarcasmo ou dupla negação o processo realizado tanto pelo programa quanto pelo grupo na análise individual foi de tratar como um bot trataria, deixando de lado o senso humano e pessoal, para obter uma maior transparência e objetividade no tratamento da informação e diminuir o viés pessoal.

   Com o objetivo de expandir o programa construído seria necessário uma maior amostragem de tweets com mais de um assunto/produto, para que a inteligência do programa conseguisse ter um repertório maior, além de novas formas e métodos para o cálculo de probabilidades com o objetivo de comparar e adequar ao melhor caso possível o melhor método de análise. Com apenas alguns dias de projeto o grupo foi capaz de realizar um programa eficiente, com espaço para aprimoramentos, mas que consegue atender ao propósito de forma efetiva, visto esse desempenho e o bom planejamento para atingir os próximos objetivos o grupo está mais que preparado para novos projetos.

   Quando tratamos de expandir os horizontes e aumentar o alcance do programa temos alguns caminhos possíveis de serem trilhados, como por exemplo, visto o contexto atual, análises sobre campanhas políticas, identificando não só por meio de tweets a percepção da população sobre determinados candidatos ou até mesmo temas de forte discussão.
